<a href="https://colab.research.google.com/github/avril-rdgz/SCEDS_TVMA/blob/naima_branch/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I. Code replication

Packages and data load

In [1]:
import pandas as pd

url="https://raw.githubusercontent.com/avril-rdgz/SCEDS_TVMA/refs/heads/main/predictors_data.csv"
df = pd.read_csv(url)
df.head()

,CRSP_SPvwx,DFY,tbl,ntis,TMS,log_DP,log_EP,lty,bm,infl,log_DY,log_DE,svar,ltr,DFR
0,-0.012417,0.0060,0.0038,0.050864,0.0175,-3.047824,-2.480303,0.0213,0.741535,0.019952,-3.056357,-0.567521,0.005718,0.003502,0.004205
1,-0.002561,0.0066,0.0038,0.046888,0.0178,-2.996390,-2.357310,0.0216,0.741117,0.004533,-2.993757,-0.639080,0.006663,0.000587,0.003818
2,-0.005630,0.0062,0.0080,0.044030,0.0133,-2.951079,-2.277102,0.0213,0.740323,0.043557,-2.957675,-0.673977,0.004086,0.009987,-0.028135
3,0.014253,0.0066,0.0095,0.029166,0.0148,-2.902206,-2.251619,0.0243,0.725326,0.018767,-2.889710,-0.650588,0.002356,-0.039832,0.022582
4,-0.014373,0.0070,0.0100,0.027165,0.0141,-2.875888,-2.176876,0.0241,0.841422,-0.000104,-2.890372,-0.699012,0.004842,0.010032,0.007850


Model

Methodology: Estimator

Methodology: Comparison